<a id="section7"></a>
## 4.7 Creating and Saving a folium Interactive Map

Now that you have seen most of the ways you can add a geodataframe to a folium map, let's create one big map that includes several of our geodataframes.

To control the display of the data layers, we will add a `folium.LayerControl`

- A `folium.LayerControl` will allow you to toggle on/off a map's visible layers. 

- In order to add a layer to the LayerControl, the layer must have value set for its `name`.

Let's take a look. 

In [ ]:
# Get our center point
ctrX = (tracts_gdf.total_bounds[0] + tracts_gdf.total_bounds[2])/2
ctrY = (tracts_gdf.total_bounds[1] + tracts_gdf.total_bounds[3])/2

# Create a new map centered on the census tract data
map6 = folium.Map(location=[ctrY, ctrX], 
                  tiles='CartoDB Positron',
                  #width=800,height=600,
                  zoom_start=10)

# Add the census tract polygons as a choropleth map
layer1=folium.Choropleth(geo_data=tracts_gdf[['GEOID','geometry']].set_index('GEOID'),
           data=tracts_gdf,
           columns=['GEOID','c_race'],
           fill_color="Reds",
           fill_opacity=0.65,
           line_color="grey", #"white",
           line_weight=1,
           line_opacity=1,
           key_on="feature.id",
           legend=True,
           legend_name="Population",
           highlight=True,
           name="census tracts"
          ).add_to(map6)

# Add the tooltip for the census tracts as its own layer
# Don't display in the Layer control!
layer2 = folium.GeoJson(tracts_gdf,
    style_function=lambda x: {'color':'transparent','fillColor':'transparent'},
    tooltip=folium.features.GeoJsonTooltip(
        fields=['GEOID','c_race' ], 
        aliases=['Tract ID', 'Population'],
        labels=True,
        localize=True
    ),
    highlight_function=lambda x: {'weight':3,'color':'white'}
).add_to(layer1.geojson)

# Add Bart lines
folium.GeoJson(bart_lines,
               name="Bart Lines",
               tooltip=folium.GeoJsonTooltip(
                   fields=['operator' ],
                   aliases=['Line operator'],
                   labels=True,
                   localize=True
               ),
              ).add_to(map6)


# Add Bart stations
folium.GeoJson(bart_stations,
               name="Bart stations",
              tooltip=folium.GeoJsonTooltip(fields=['ts_locatio' ], 
                   aliases=['Stop Name'],
                   labels=True,
                   localize=True
               ),
              ).add_to(map6)

# ADD LAYER CONTROL
folium.LayerControl(collapsed=False).add_to(map6)

map6  # show map

<div style="display:inline-block;vertical-align:top;">
    <img src="https://image.flaticon.com/icons/svg/87/87705.svg" width="30" align=left > 
</div>  
<div style="display:inline-block;">

#### Questions
</div>

1. Take a look at the help docs `folium.LayerControl?`. What parameter would move the location of the LayerControl? What parameter would allow it to be closed by default?

2. Take a look at the way we added `layer2` above (this has the census tract tooltips). How has the code we use to add the layer to the map changed? Why do you think we made this change?

In [ ]:
# Uncomment to view
#folium.LayerControl?

### Saving to an html file

By saving our map to a html we can use it later as something to add to a website or email to a colleague.

You can save any of the maps you have in the notebook using this syntax:

> map_name.save("file_name.html")

Let's try that.

In [ ]:
map6.save('../outdata/bartmap.html')

Find your html file on your computer and double-click on it to open it in a browser.

#### Exercise

Using the code that we used above to make the `berkeley map` and the BART map (`map5`) make an interactive map showing
- Oakland tracts, colored by median rent
- Overlaying permit locations, sized by the number of approved permits.
- Add tooltips to each layer

In [ ]:
# Your code here

*Click here for answers*

<!---

# SOLUTION

# read in the housing app permit data
permit_ha_parcel_gdf = gpd.read_file("../notebook_data/outdata/Permit_HousingApp_Parcel_Merge_Oakland.geojson")

# make a point goedataframe
housingapp_parcel_point_gdf = gpd.GeoDataFrame(permit_ha_parcel_gdf.drop('geometry',axis=1), 
                            geometry=permit_ha_parcel_gdf.centroid)

# drop any rows where the geom is null
housingapp_parcel_point_gdf=housingapp_parcel_point_gdf[~housingapp_parcel_point_gdf.geometry.isna()]

# Take a look at the columns
housingapp_parcel_point_gdf.columns


# SOLUTION

# Create our basemap centered
map5 = folium.Map(location=[oakland.centroid.y.mean(), oakland.centroid.x.mean()], 
                  tiles='CartoDB Positron',
                  #width=800,height=1000,
                  zoom_start=12,
                  name="Basemap")

# Oakland tracts colored by median rent
folium.Choropleth(geo_data=tracts_gdf[['GEOID','geometry']].set_index('GEOID'),
           data=tracts_gdf,
           columns=['GEOID','med_rent'],
           fill_color="Blues",
           fill_opacity=0.65,
           line_color="grey", #"white",
           line_weight=1,
           line_opacity=1,
           key_on="feature.id",
           legend=True,
           legend_name="Median Rent",
           highlight=True
          ).add_to(map5)

# Add the oakland boundary
folium.GeoJson(data=oakland,
                   name='Oakland',smooth_factor=2,
                   style_function=lambda x: {'color':'black','opacity':1,'fillColor':'transparent','weight':3},
                   ).add_to(map5)


# Add tool tips to the tracts  layer
folium.GeoJson(tracts_gdf,
    style_function=lambda x: {'color':'transparent','fillColor':'transparent'},
    tooltip=folium.features.GeoJsonTooltip(
        fields=['GEOID','med_rent' ], 
        aliases=['Tract ID', 'Median Rent'],
        labels=True,
        localize=True
    ),
    highlight_function=lambda x: {'weight':3,'color':'white'}
).add_to(map5)

housingapp_parcel_point_gdf.apply(lambda row:
                        folium.CircleMarker(
                                  location=[row['geometry'].y, row['geometry'].x],
                                  radius=(row['approved']+20)/10,
                                  color='purple',
                                  fill=True,
                                  fill_color='orange',
                                  tooltip = "Permit Location: %s<br>Approved Permits: %s" % (row['address'], row['approved'])    
                                 ).add_to(map5), axis=1)



# COULD ALSO Add permit locations using a for loop!
# for index, permit in housingapp_parcel_point_gdf.iterrows():
#     nice_tip = 
#     folium.CircleMarker(
#         location= [permit['geometry'].y, permit['geometry'].x],
#         radius =  (permit['approved']+20)/10,
#         tooltip = nice_tip,
#         color='purple',
#         fill=True,
#         fill_color='orange'
# ).add_to(map5)

map5  # show map
--->

In [ ]:
<a id="section7"></a>
## 4.7 Creating and Saving a folium Interactive Map

Now that you have seen most of the ways you can add a geodataframe to a folium map, let's create one big map that includes several of our geodataframes.

To control the display of the data layers, we will add a `folium.LayerControl`

- A `folium.LayerControl` will allow you to toggle on/off a map's visible layers. 

- In order to add a layer to the LayerControl, the layer must have value set for its `name`.

Let's take a look. 

# Get our center point
ctrX = (tracts_gdf.total_bounds[0] + tracts_gdf.total_bounds[2])/2
ctrY = (tracts_gdf.total_bounds[1] + tracts_gdf.total_bounds[3])/2

# Create a new map centered on the census tract data
map6 = folium.Map(location=[ctrY, ctrX], 
                  tiles='CartoDB Positron',
                  #width=800,height=600,
                  zoom_start=10)

# Add the census tract polygons as a choropleth map
layer1=folium.Choropleth(geo_data=tracts_gdf[['GEOID','geometry']].set_index('GEOID'),
           data=tracts_gdf,
           columns=['GEOID','c_race'],
           fill_color="Reds",
           fill_opacity=0.65,
           line_color="grey", #"white",
           line_weight=1,
           line_opacity=1,
           key_on="feature.id",
           legend=True,
           legend_name="Population",
           highlight=True,
           name="census tracts"
          ).add_to(map6)

# Add the tooltip for the census tracts as its own layer
# Don't display in the Layer control!
layer2 = folium.GeoJson(tracts_gdf,
    style_function=lambda x: {'color':'transparent','fillColor':'transparent'},
    tooltip=folium.features.GeoJsonTooltip(
        fields=['GEOID','c_race' ], 
        aliases=['Tract ID', 'Population'],
        labels=True,
        localize=True
    ),
    highlight_function=lambda x: {'weight':3,'color':'white'}
).add_to(layer1.geojson)

# Add Bart lines
folium.GeoJson(bart_lines,
               name="Bart Lines",
               tooltip=folium.GeoJsonTooltip(
                   fields=['operator' ],
                   aliases=['Line operator'],
                   labels=True,
                   localize=True
               ),
              ).add_to(map6)


# Add Bart stations
folium.GeoJson(bart_stations,
               name="Bart stations",
              tooltip=folium.GeoJsonTooltip(fields=['ts_locatio' ], 
                   aliases=['Stop Name'],
                   labels=True,
                   localize=True
               ),
              ).add_to(map6)

# ADD LAYER CONTROL
folium.LayerControl(collapsed=False).add_to(map6)

map6  # show map

<div style="display:inline-block;vertical-align:top;">
    <img src="https://image.flaticon.com/icons/svg/87/87705.svg" width="30" align=left > 
</div>  
<div style="display:inline-block;">

#### Questions
</div>

1. Take a look at the help docs `folium.LayerControl?`. What parameter would move the location of the LayerControl? What parameter would allow it to be closed by default?

2. Take a look at the way we added `layer2` above (this has the census tract tooltips). How has the code we use to add the layer to the map changed? Why do you think we made this change?

# Uncomment to view
#folium.LayerControl?

### Saving to an html file

By saving our map to a html we can use it later as something to add to a website or email to a colleague.

You can save any of the maps you have in the notebook using this syntax:

> map_name.save("file_name.html")

Let's try that.

map6.save('../outdata/bartmap.html')

Find your html file on your computer and double-click on it to open it in a browser.

#### Exercise

Using the code that we used above to make the `berkeley map` and the BART map (`map5`) make an interactive map showing
- Oakland tracts, colored by median rent
- Overlaying permit locations, sized by the number of approved permits.
- Add tooltips to each layer

# Your code here

*Click here for answers*

<!---

# SOLUTION

# read in the housing app permit data
permit_ha_parcel_gdf = gpd.read_file("../notebook_data/outdata/Permit_HousingApp_Parcel_Merge_Oakland.geojson")

# make a point goedataframe
housingapp_parcel_point_gdf = gpd.GeoDataFrame(permit_ha_parcel_gdf.drop('geometry',axis=1), 
                            geometry=permit_ha_parcel_gdf.centroid)

# drop any rows where the geom is null
housingapp_parcel_point_gdf=housingapp_parcel_point_gdf[~housingapp_parcel_point_gdf.geometry.isna()]

# Take a look at the columns
housingapp_parcel_point_gdf.columns


# SOLUTION

# Create our basemap centered
map5 = folium.Map(location=[oakland.centroid.y.mean(), oakland.centroid.x.mean()], 
                  tiles='CartoDB Positron',
                  #width=800,height=1000,
                  zoom_start=12,
                  name="Basemap")

# Oakland tracts colored by median rent
folium.Choropleth(geo_data=tracts_gdf[['GEOID','geometry']].set_index('GEOID'),
           data=tracts_gdf,
           columns=['GEOID','med_rent'],
           fill_color="Blues",
           fill_opacity=0.65,
           line_color="grey", #"white",
           line_weight=1,
           line_opacity=1,
           key_on="feature.id",
           legend=True,
           legend_name="Median Rent",
           highlight=True
          ).add_to(map5)

# Add the oakland boundary
folium.GeoJson(data=oakland,
                   name='Oakland',smooth_factor=2,
                   style_function=lambda x: {'color':'black','opacity':1,'fillColor':'transparent','weight':3},
                   ).add_to(map5)


# Add tool tips to the tracts  layer
folium.GeoJson(tracts_gdf,
    style_function=lambda x: {'color':'transparent','fillColor':'transparent'},
    tooltip=folium.features.GeoJsonTooltip(
        fields=['GEOID','med_rent' ], 
        aliases=['Tract ID', 'Median Rent'],
        labels=True,
        localize=True
    ),
    highlight_function=lambda x: {'weight':3,'color':'white'}
).add_to(map5)

housingapp_parcel_point_gdf.apply(lambda row:
                        folium.CircleMarker(
                                  location=[row['geometry'].y, row['geometry'].x],
                                  radius=(row['approved']+20)/10,
                                  color='purple',
                                  fill=True,
                                  fill_color='orange',
                                  tooltip = "Permit Location: %s<br>Approved Permits: %s" % (row['address'], row['approved'])    
                                 ).add_to(map5), axis=1)



# COULD ALSO Add permit locations using a for loop!
# for index, permit in housingapp_parcel_point_gdf.iterrows():
#     nice_tip = 
#     folium.CircleMarker(
#         location= [permit['geometry'].y, permit['geometry'].x],
#         radius =  (permit['approved']+20)/10,
#         tooltip = nice_tip,
#         color='purple',
#         fill=True,
#         fill_color='orange'
# ).add_to(map5)

map5  # show map
--->